In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import csv as csv
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from citipy import citipy

In [45]:
df_1=pd.read_csv("Resources/Milton_Avg_Rental_Price_updated.csv", encoding='ISO-8859-1')
df_2=pd.read_csv("Resources/Milton_Avg_Sale_Price_updated.csv", encoding='ISO-8859-1')
df_3=pd.read_csv("Resources/Milton_Median_Rental_Price_updated.csv", encoding='ISO-8859-1')
df_4=pd.read_csv("Resources/Milton_Median_Sale_Price_updated.csv", encoding='ISO-8859-1')

,Primary Year,"Sale Price, Median"
0,2016,"$621,000"
1,2017,"$705,000"
2,2018,"$675,000"
3,2019,"$725,000"
4,2020,"$810,786"
5,2021,"$990,000"
6,2022,"$1,100,000"
7,2023,"$1,015,000"


In [32]:
milton_df_2021 = pd.read_csv("Resources/CensusProfile2021-ProfilRecensement2021-20231206084412.csv", encoding='ISO-8859-1')
milton_df

,Topic,Characteristic,Total
0,Population and dwellings,"Population, 2021",132979.0
1,Population and dwellings,"Population, 2016",110128.0
2,Population and dwellings,"Population percentage change, 2016 to 2021",20.7
3,Population and dwellings,Total private dwellings,41000.0
4,Population and dwellings,Private dwellings occupied by usual residents,40038.0
...,...,...,...
223,Labour force status,Unemployed,8035.0
224,Labour force status,Not in the labour force,28790.0
225,Labour force status,Participation rate,71.1
226,Labour force status,Employment rate,63.0


In [23]:
milton_df.columns

Index(['Topic', 'Characteristic', 'Total'], dtype='object')

In [27]:
#milton_housing_df = pd.read_csv("Resources/TableExport (1).csv", encoding='ISO-8859-1',sep=';')
#milton_housing_df.columns

In [30]:
milton_df_2016 = pd.read_csv("Resources/CensusProfile2016-ProfilRecensement2016-20231207084508.csv", encoding='ISO-8859-1')
milton_df_2016

,Topic,Characteristics,Unnamed: 2,Total
0,Population and dwellings,Population; 2016,NaN,110128.0
1,Population and dwellings,Population; 2011,NaN,84362.0
2,Population and dwellings,Population percentage change; 2011 to 2016,NaN,30.5
3,Population and dwellings,Total private dwellings,NaN,35537.0
4,Population and dwellings,Private dwellings occupied by usual residents,NaN,34257.0
...,...,...,...,...
2242,Mobility status - Place of residence 5 years ago,Migrants,NaN,31220.0
2243,Mobility status - Place of residence 5 years ago,Internal migrants,NaN,25720.0
2244,Mobility status - Place of residence 5 years ago,Intraprovincial migrants,NaN,24440.0
2245,Mobility status - Place of residence 5 years ago,Interprovincial migrants,NaN,1280.0


In [39]:
milton_df = pd.merge(milton_df_2016, milton_df_2021, how='left', left_index=True, right_index=True)
del milton_df['Unnamed: 2']
milton_df

,Topic_x,Characteristics,Total_x,Topic_y,Characteristic,Total_y
0,Population and dwellings,Population; 2016,110128.0,Population and dwellings,"Population, 2021",132979.0
1,Population and dwellings,Population; 2011,84362.0,Population and dwellings,"Population, 2016",110128.0
2,Population and dwellings,Population percentage change; 2011 to 2016,30.5,Population and dwellings,"Population percentage change, 2016 to 2021",20.7
3,Population and dwellings,Total private dwellings,35537.0,Population and dwellings,Total private dwellings,41000.0
4,Population and dwellings,Private dwellings occupied by usual residents,34257.0,Population and dwellings,Private dwellings occupied by usual residents,40038.0
...,...,...,...,...,...,...
2242,Mobility status - Place of residence 5 years ago,Migrants,31220.0,NaN,NaN,NaN
2243,Mobility status - Place of residence 5 years ago,Internal migrants,25720.0,NaN,NaN,NaN
2244,Mobility status - Place of residence 5 years ago,Intraprovincial migrants,24440.0,NaN,NaN,NaN
2245,Mobility status - Place of residence 5 years ago,Interprovincial migrants,1280.0,NaN,NaN,NaN


In [48]:
droped_milton_df = milton_df.dropna()
droped_milton_df.head(50)

,Topic_x,Characteristics,Total_x,Topic_y,Characteristic,Total_y
0,Population and dwellings,Population; 2016,110128.00,Population and dwellings,"Population, 2021",132979.00
1,Population and dwellings,Population; 2011,84362.00,Population and dwellings,"Population, 2016",110128.00
2,Population and dwellings,Population percentage change; 2011 to 2016,30.50,Population and dwellings,"Population percentage change, 2016 to 2021",20.70
3,Population and dwellings,Total private dwellings,35537.00,Population and dwellings,Total private dwellings,41000.00
4,Population and dwellings,Private dwellings occupied by usual residents,34257.00,Population and dwellings,Private dwellings occupied by usual residents,40038.00
5,Population and dwellings,Population density per square kilometre,303.20,Population and dwellings,Population density per square kilometre,365.50
6,Population and dwellings,Land area in square kilometres,363.22,Population and dwellings,Land area in square kilometres,363.83
7,Age characteristics,Total - Age groups and average age of the popu...,110125.00,Age characteristics,Total - Age groups of the population - 100% data,132980.00
8,Age characteristics,0 to 14 years,28405.00,Age characteristics,0 to 14 years,31980.00
9,Age characteristics,0 to 4 years,9315.00,Age characteristics,0 to 4 years,8875.00
